In [11]:
import os
import sys
import ccxt
import pandas as pd
import collections

file = "C:/Users/Carter Carlson/Documents/Excel references/secret.csv"
api = pd.read_csv(file)

binance = ccxt.binance({
    'options':{'adjustForTimeDifference':True},
    'apiKey': api['apiKey'][0],
    'secret': api['secret'][0]
})

balance = binance.fetchBalance()
coins = ['BTC', 'ETH', 'XRP', 'OMG']

data = pd.DataFrame(columns=['symbol', 'quantity', 'price', 'dollar_value'])
btc_price = float(binance.fetch_ticker('BTC/USDT')['info']['lastPrice'])
total_value = 0

for coin in coins:
    quantity = balance[coin]['total']
    
    if coin == 'BTC':
        price = btc_price
    else:
        btc_ratio = float(binance.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
        price = btc_ratio * btc_price

    dollar_value = quantity * price
    total_value += dollar_value

    data = data.append({'symbol': coin,
                        'quantity': quantity,
                        'price': price,
                        'dollar_value': dollar_value},
                      ignore_index=True)

data['weight'] = list(map(lambda x: x / total_value, data['dollar_value']))

In [ ]:
# test here for nonexistent ticker
try:
    binance.fetch_ticker('BTC/XRP')['info']['lastPrice']
    print('A works')
except:
    binance.fetch_ticker('XRP/BTC')['info']['lastPrice']
    print('B works')

In [ ]:
data['weight'] = list(map(lambda x: x / total_value, data['dollar_value']))
data[data['weight'] > .2]

index_max = data['weight'].index(max(data['weight']))
index_min = data['weight'].index(min(data['weight']))

if data['weight'][index_max] - data['weight'][index_min] > .05:
    weight_to_sell = (data['weight'][index_max] - data['weight'][index_min]) / 2
    
  
    try:
        side = 'sell'
        test_ticker = data['symbol'][index_max] + '/' + data['symbol'][index_min]
        selling_coin_quantity = (weight_to_sell * total_value) / data['price'][index_max]
        # note: if fails at selling_coin_quantity, move test_price outside try
        test_price = float(binance.fetch_ticker(test_ticker)['info']['lastPrice'])

    except:
        side = 'buy'
        test_ticker = data['symbol'][index_min] + '/' + data['symbol'][index_max]
        selling_coin_quantity = weight to sell * total_value / data['price'][index_min]

        test_price = float(binance.fetch_ticker(test_ticker)['info']['lastPrice'])
        
    binance.create_order(test_ticker, 'market', side, selling_coin_quantity)

# Note: see difference between using market or limit orders-
#     a. Does limit have automatic setting?
#     b. Different fees for limit or market?
#     c. What happens if limit doesn't fill